<a href="https://colab.research.google.com/github/Valgueiro/dataScience_project/blob/master/Projeto_1_dataScience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalar bibliotecas necessárias

In [ ]:
# Por enquando nenhuma.

# Analisandos os dados da temporada 2019/20 da NFL

### Coletando dados

Utilizando https://www.kaggle.com/dvc415/nfl-2019-weekly-player-stats



In [ ]:
import numpy as np
import pandas as pd

In [ ]:
qbs = pd.read_csv("https://raw.githubusercontent.com/Valgueiro/dataScience_project/master/data/nfl_2019_qb_weekly_stats.csv")
rbs = pd.read_csv("https://raw.githubusercontent.com/Valgueiro/dataScience_project/master/data/nfl_2019_rb_weekly_stats.csv")
wrs = pd.read_csv("https://raw.githubusercontent.com/Valgueiro/dataScience_project/master/data/nfl_2019_wr_weekly_stats.csv")
rbs.head()

,Rank,Player,Position,Age,Date,League,Team,Opp,Result,Game,Week,Day,Attempts,Rush_Yards,Yards_per_Attempt,Rush_TD
0,1,Marlon Mack,RB,23.185,2019-09-08,NFL,IND,LAC,L 24-30,1,1,Sun,25,174,6.96,1
1,2,Christian McCaffrey,RB,23.093,2019-09-08,NFL,CAR,LAR,L 27-30,1,1,Sun,19,128,6.74,2
2,3,Saquon Barkley,RB,22.211,2019-09-08,NFL,NYG,DAL,L 17-35,1,1,Sun,11,120,10.91,0
3,4,Dalvin Cook,RB,24.029,2019-09-08,NFL,MIN,ATL,W 28-12,1,1,Sun,21,111,5.29,2
4,5,Mark Ingram,RB,29.261,2019-09-08,NFL,BAL,MIA,W 59-10,1,1,Sun,14,107,7.64,2
